In [1]:
import sys
import xlrd
import nltk
import numpy as np
import pandas as pd
import sqlite3 as sq

In [9]:

def needEscape(l):
    if l.find(u'人名') > 0:
        return True
    if l.find(u'货币基本单位') > 0:
        return True
    if l.find('abbr') > -1:
        return True
    return False

In [2]:
def xls2list(fname):

    data = xlrd.open_workbook(fname + '.xls')
    table = data.sheets()[0]
    nrows = table.nrows # 获取表的行数
    l = ['word', 'level', 'lenovo', 'etyma', 'meanZh', 'meanEn', 'example', 'phonetic', 'test']

    content = []

    for i in range(1, nrows): # 循环逐行打印
        if i == 0: # 跳过第一行
            continue
        row = table.row_values(i)[:len(l)]
        if len(row) > len(l):
            print len(row), row
            sys.exit()
        content.append(row)

    print len(content)
    return content, l

def list2df(para):
    content, l = para
    data = pd.DataFrame(content, columns=l)

    data['selfLenovo'] = ""
    data['level'] = 0
    return data

def gloss2test(data):

    test = {'CET4':'四级', 'CET6':'六级', 'TOEFL':'托福', 
            'GRE':'GRE', 'GRE3000':'GRE-再要你命3000', 
            'GREmagoosh':'GRE-Magoosh'}

    f = file("books.txt", "w")

    for t in test:
        subset = data[data['test'].str.contains(t)]
        con  = sq.connect(t.lower() + '.db')
        subset[subset.columns[:-1]].to_sql('word', con)
        con.close()

        m = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime(os.path.getmtime('gloss.xls')))
        s = "%s http://7xt8es.com1.z0.glb.clouddn.com/naodong/word/%s.db %d(%s更新)" % (test[t], t.lower(), len(subset), m)

        f.write(s)

        f.write('\n')

    f.close()

    return len(content)


In [3]:
data = list2df(xls2list('gloss2'))

19211


In [4]:
len(data[(data['phonetic'] == '')])

668

In [33]:
test = YoudaoSpider('abjectly')
sample = test.get_result()
result = transform(sample)
printDeep(result, 2)
result['phonetic']

u"'\xe6bd\u0292ektli"

In [62]:
import re
import json
import os
import requests
import webbrowser
#from termcolor import colored
from bs4 import BeautifulSoup


# In[17]:

def printDeep(item, deep):
    if isinstance(item, (str, bool, int, float)):
        print ' '*deep, item
    elif isinstance(item, (list, tuple)):
        print ' '*deep, 'a list'
        for e in item:
            if isinstance(e, (str, bool, int, float, unicode)):
                print ' '*(deep+4), e
            else:
                printDeep(e, deep+4)
    elif isinstance(item, dict):
        print ' '*deep, 'a dict'
        for e in item:
            print ' '*(deep+4), e, ':', item[e]
            if not isinstance(item[e], (str, bool, int, float)):
                printDeep(item[e], deep+4)


# In[45]:

def addBilingual(example):
    bilingual = example.find(id='bilingual')
    if not bilingual:
        return None
    ul = bilingual.find_all('li')
    b = []
    for l in ul:
        t = l.find_all('p')
        en = ''.join(e.string if e.string else e.text for e in t[0].find_all('span'))
        zh = ''.join(e.string if e.string else e.text for e in t[1].find_all('span'))
        b.append(zh + ':' + en)
    return b


# In[75]:

def addExample(example, name):
    items = example.find(id=name)
    if not items:
        return None
    ul = items.find_all('li')
    b = []
    for l in ul:
        en = l.find('p').text
        b.append(':' + en.strip())
    return b


# In[91]:


class YoudaoSpider:
    """
    通过有道获取单词解释, 以及展示查询结果
    """

    params = {
        'keyfrom': 'longcwang',
        'key': '131895274',
        'type': 'data',
        'doctype': 'json',
        'version': '1.1',
        'q': 'query'
    }
    api_url = u'http://fanyi.youdao.com/openapi.do'
    voice_url = u'http://dict.youdao.com/dictvoice?type=2&audio={word}'
    web_url = u'http://dict.youdao.com/search?keyfrom=dict.top&q='
    translation_url = u'http://fanyi.youdao.com/translate?keyfrom=dict.top&i='

    error_code = {
        0: u'正常',
        20: u'要翻译的文本过长',
        30: u'无法进行有效的翻译',
        40: u'不支持的语言类型',
        50: u'无效的key',
        60: u'无词典结果，仅在获取词典结果生效'
    }

    result = {
        "query": "",
        "errorCode": 0,
    }

    def __init__(self, word):
        self.word = word

    def get_result(self, use_api=False):
        """
        获取查询结果
        :param use_api:是否使用有道API, 否则解析web版有道获取结果
        :return:与有道API返回的json数据一致的dict
        """
        if use_api:
            self.params['q'] = self.word
            r = requests.get(self.api_url, params=self.params)
            r.raise_for_status()    # a 4XX client error or 5XX server error response
            self.result = r.json()
        else:
            r = requests.get(self.web_url + self.word)
            r.raise_for_status()
            self.parse_html(r.text)
        return self.result

    def parse_html(self, html):
        """
        解析web版有道的网页
        :param html:网页内容
        :return:result
        """
        soup = BeautifulSoup(html, "lxml")
        root = soup.find(id='results-contents')
        
        if not root:
            self.result = {'query': '', 'example': '', 'basic':{'explains':''}}
            return

        # query 搜索的关键字
        keyword = root.find(class_='keyword')
        if not keyword:
            self.result['query'] = self.word
        else:
            self.result['query'] = unicode(keyword.string)

        # 基本解释
        basic = root.find(id='phrsListTab')
        if basic:
            trans = basic.find(class_='trans-container')
            if trans:
                self.result['basic'] = {}
                self.result['basic']['explains'] = [unicode(tran.string) for tran in trans.find_all('li')]
                # 中文
                if len(self.result['basic']['explains']) == 0:
                    exp = trans.find(class_='wordGroup').stripped_strings
                    self.result['basic']['explains'].append(' '.join(exp))

                # 音标
                phons = basic(class_='phonetic', limit=2)
                if len(phons) == 2:
                    self.result['basic']['uk-phonetic'], self.result['basic']['us-phonetic'] =                         [unicode(p.string)[1:-1] for p in phons]
                elif len(phons) == 1:
                    self.result['basic']['phonetic'] = unicode(phons[0].string)[1:-1]
        # 英文译义
        # meanEn = root.find(id='tEETrans')
        # 翻译
        if 'basic' not in self.result:
            self.result['translation'] = self.get_translation(self.word)

        # 网络释义(短语)
        web = root.find(id='webPhrase')
        if web:
            self.result['web'] = [
                {
                    'key': unicode(wordgroup.find(class_='search-js').string).strip(),
                    'value': [v.strip() for v in unicode(wordgroup.find('span').next_sibling).split(';')]
                } for wordgroup in web.find_all(class_='wordGroup', limit=4)
            ]
            
        example = root.find(id='examplesToggle')
        self.result['example'] = []
        if not example:
            return
        
        bi = addBilingual(example)

        if bi:
            self.result['example'] += bi
        other = addExample(example, 'originalSound')
        if other:
            self.result['example'] += other
        other = addExample(example, 'authority')
        if other:
            self.result['example'] += other        

    def get_translation(self, word):
        """
        通过web版有道翻译抓取翻译结果
        :param word:str 关键字
        :return:list 翻译结果
        """
        r = requests.get(self.translation_url+word)
        if r.status_code != requests.codes.ok:
            return None

        pattern = re.compile(r'"translateResult":\[(\[.+\])\]')
        m = pattern.search(r.text)
        result = json.loads(m.group(1))
        return [item['tgt'] for item in result]

    @classmethod
    def get_voice(cls, word):
        voice_file = os.path.join(VOICE_DIR, word+'.mp3')
        if not os.path.isfile(voice_file):
            r = requests.get(cls.voice_url.format(word=word))
            with open(voice_file, 'wb') as f:
                f.write(r.content)
        return voice_file


# In[103]:

# word	level	lenovo	etyma	meanZh	meanEn	example	phonetic
def transform(sample):
    result = {'word': sample['query'], 'example': '\n'.join(sample['example']),
             'meanZh': '\n'.join(sample['basic']['explains'])}
    if 'phonetic' in sample['basic']:
        result['phonetic'] = sample['basic']['phonetic']
    elif 'uk-phonetic' in sample['basic'] and 'us-phonetic' in sample['basic']:
        result['phonetic'] = u'英:[' + sample['basic']['uk-phonetic'] + u'] 美:[' +            sample['basic']['uk-phonetic'] + u']'
    return result


# In[105]:


In [63]:
demo = YoudaoSpider('create')
sample = demo.get_result()
result = transform(sample)

In [59]:

def crawl(x):
    global c
    c += 1
    print c
    demo = YoudaoSpider(x[0])
    sample = demo.get_result()
    result = transform(sample)

    return result['example'] if 'example' in result else ''

In [62]:
c = 0
data[data['example'] == ''][:4].apply(crawl, 1)

2     鍦ㄨ嫳璇�噷鎴戜滑鍙�畠涓轰竴涓�畻鐩樸�:In English we call it an abacus...
10    鍦�娓呭崟 1涓�垜澶ч噺浣跨敤浜嗘�缂╁啓銆�I used this abbreviation hea...
13    鎵�弿浠栫殑鑳搁儴锛岃吂閮ㄥ拰楠ㄧ泦涔熸病鏈変粈涔堝彂鐜般�:Scans of his chest, abdom...
23    鎴戜笉鑳藉�蹇嶆�楠�:I can't abide dishonesty!\n浣犳�鑳藉�蹇嶄粬?:Ho...
dtype: object

In [54]:
data.loc[data['phonetic'].str[0] == '/', 'phonetic'] = data[data['phonetic'].str[0] == '/'].apply(crawl, 1)

In [53]:
len(data[data['phonetic'].str[0] == '/'])

1292

In [55]:
len(data[data['phonetic'].str[0] == '/'])

0

In [64]:
#data[(data['example'] == '')]
data.loc[data['example'] == '', 'example'] = data[data['example'] == ''].apply(crawl, 1)

In [65]:
len(data[(data['example'] == '')])

7

In [20]:
test = {'CET4':'四级', 'CET6':'六级', 'TOFEL':'托福', 
            'GRE':'GRE', 'GRE3000':'GRE-再要你命3000', 
            'GREmagoosh':'GRE-Magoosh'}

for t in test:
    if t == 'CET6':
        subset = data[(data['test'].str.contains(t))&(~data['test'].str.contains('CET4'))]
    else:
        subset = data[(data['test'].str.contains(t))]
    print t, len(subset)

CET4 4510
TOFEL 2081
CET6 11171
GRE3000 3060
GRE 7972
GREmagoosh 1010


In [77]:
t = data[data['test'].str.contains('GREmagoosh')]
print len(t)

1010


In [78]:
a = data[data['test'].str.contains('GRE3000')]
print len(a)

2719


In [80]:
p = list2df(xls2list('another-3000'))

3094


In [81]:
tw = set(t['word'])
pw = set(p['word'])

In [82]:
len(tw.intersection(pw))

608

In [83]:
d = pw.difference(tw)
print len(d)

2480


In [84]:
data.loc[data['word'].isin(d), 'test'] += ' GRE3000'

In [40]:
data.columns

Index([u'word', u'level', u'lenovo', u'etyma', u'meanZh', u'meanEn',
       u'example', u'phonetic', u'test'],
      dtype='object')

In [85]:
len(data)

19211

In [43]:
len(data.loc[data['word'].isin(d), 'word'])

0

In [90]:
writer = pd.ExcelWriter('gloss4.xls')
data.to_excel(writer, index = False)
writer.save()

In [47]:
len(data[data['word'].isin(d)])

369

In [68]:
len(data.loc[:, 'example'])

19211

In [6]:
import string

In [8]:
def is_chinese(uchar):
    """判断一个unicode是否是汉字"""
    if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
        return True
    else:
        return False

In [9]:
x = u"中"
is_chinese(x)

True

In [13]:
x = '123132354523'
x.find('12')

0

In [23]:
def adjustExample(x):
    lines = x.strip().split('\n')
    result = []
    for l in lines:
        if l.find('||') > -1:
            result.append(l)
            continue
            
        parts = l.split(':')
        
        if len(parts) == 1:
            continue
        if len(parts) > 2:
            c = ':'.join(parts[1:]).strip()
        else:
            c = parts[1].strip()
        words = c.split()
        p = len(words)
        for w in words:
            if is_chinese(w[0]):
                p = words.index(w)
                break
            
        sentence_en = ' '.join(words[:p])
        sentence_ch = ' '.join(words[p:])
        if not parts[0]:
            z = sentence_ch
        else:
            z = parts[0]
            if sentence_ch:
                z += '#' + sentence_ch
        result.append(z + ':' + sentence_en)
    result.sort(key = lambda x:len(x), reverse=True)
    return '\n'.join(result)

In [24]:
data['example'][:5].apply(adjustExample)

0    :We call it a function. That is to say it has ...
1    :Curt Onalfo's playing career included many ap...
2    为了改善学生日渐衰退的计算能力，日本的一些学校重新使用起一个古老的计算工具-算盘。:In r...
3    v. 放纵#感情用事，abandon herself to a life of comple...
4    v. 降低（地位、职位、威望或尊严）#不愿意屈就自己去承认一个莫须有的罪名:was unwi...
Name: example, dtype: object

In [25]:
data.loc[:, 'example'] = data['example'].apply(adjustExample)

In [5]:
x = ['x']
':'.join(x)

'x'

In [50]:
x = 'adfsd,fsdofksj;fsafsd'
import re
re.split('[,;]', x)

['adfsd', 'fsdofksj', 'fsafsd']

In [88]:
import re
def adjustMeanZh(x):
    global c
    
    if not x[4] or x[4][0] not in string.ascii_letters:
        c += 1
        print x[0]
        #demo = YoudaoSpider(x[4])
        #sample = demo.get_result()
        #result = transform(sample)
        #return result['meanZh']
        return ''
        
    #result = '\n'.join(l.replace('a.', 'adj.') for l in x[4].strip().split('\n')
    #                   if l and l[0] in string.ascii_letters and l.find('.') > -1 and l.find(u'人名') == -1)    
    # result = '\n'.join(l.replace('a.', 'adj.') for l in x[4].strip().split('\n') if l.find(u'人名') == -1)    
    return x[4]
        

In [89]:
c = 0
x = data.apply(adjustMeanZh, 1)
print c

apocrypha
bolted
condole  with
conduce  to
coterminous
dainty
drabness
epistle
expostulate
gargoyle
haemophilia
haemostat
hauteur
hecatomb
hors d' oeuvre
hot  water
imbue  with
incipience
infallibility
jigsaw  puzzle
June
mercedes
misattribute
mistimed
muniments
phillip
prometheus
purlieus
saith
snips
spleenish
sullivan
susceptibilities
tined
unfailing
35


19211

In [87]:
c = 0
data.loc[:, 'meanZh'] = data.apply(adjustMeanZh, 1)
print c

apocrypha
bolted
condole  with
conduce  to
coterminous
dainty
drabness
epistle
expostulate
gargoyle
haemophilia
haemostat
hauteur
hecatomb
hors d' oeuvre
hot  water
imbue  with
incipience
infallibility
jigsaw  puzzle
June
mercedes
misattribute
mistimed
muniments
phillip
prometheus
purlieus
saith
snips
spleenish
sullivan
susceptibilities
tined
unfailing
35


In [78]:
print c

421


In [ ]:
data[:3]

In [41]:
c = 0
x = data
print len(x)
print c

SyntaxError: invalid syntax (<ipython-input-41-37da20cff78f>, line 2)

In [ ]:
.apply(adjustMeanZh, 1)